<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 2*

# Sprint Challenge - Neural Network Foundations

Table of Problems

1. [Defining Neural Networks](#Q1)
2. [Perceptron on XOR Gates](#Q2)
3. [Multilayer Perceptron](#Q3)
4. [Keras MMP](#Q4)

<a id="Q1"></a>
## 1. Define the following terms:

- **Neuron:** 
- **Input Layer:** 
- **Hidden Layer:** 
- **Output Layer:**
- **Activation:**
- **Backpropagation:**


neuron: basic building block of a neural network; accepts an input, calculates the weighted summation of inputs and passes through an activation function before firing

input layer: layer that receives input or X (feature) variables

hidden layer: either one or multiple, defines "deep learning" and cannot be accessed except through the input layer;
performs transformation on data

output layer: final layer returning "y" predictions

activation: varying functions that decides whether or not a perceptron fires; takes numbers and compresses them into a smaller range of values to determine what information is carried to the next node

backpropagation: method to update the weights of a neural net based upon the calculated errors of predictions

## 2. Perceptron on XOR Gates <a id="Q2"></a>

The XOr, or “exclusive or”, problem is a classic problem in ANN research. It is the problem of using a neural network to predict the outputs of XOr logic gates given two binary inputs. An XOr function should return a true value if the two inputs are not equal and a false value if they are equal. Create a perceptron class that can model the behavior of an AND gate. You can use the following table as your training data:

|x1	|x2 | y |
|---|---|---|
| 0 | 0 | 0 |
| 0 | 1 | 1 |
| 1 | 1 | 0 |
| 1 | 0 | 1 |


In [3]:
import numpy as np

In [1]:
class Perceptron(object): 
    
    def __init__(self,n_inputs, threshold=100, learning_rate=.01):
        self.threshold = threshold 
        self.learning_rate = learning_rate
        self.weights = np.zeros(n_inputs+1)
    
    def predict(self, inputs):
        summation = np.dot(inputs, self.weights[1:])
        if summation > 0:
            activation = 1
        else:
            activation = 0
        return activation
    
    def train(self, training_inputs, labels):
        for _ in range(self.threshold):
            for inputs, label in zip(training_inputs, labels):
                prediction = self.predict(inputs)
                self.weights[1:] += self.learning_rate * (label - prediction) * prediction
                self.weights[0] += self.learning_rate * (label - prediction)

In [4]:
inputs = np.array([[1,1,1],
                 [0,0,0],
                 [1,1,0],
                 [1,0,1]])

labels = np.array([[0],[1],[0],[1]])


In [5]:
perceptron = Perceptron(n_inputs=3)
perceptron.train(training_inputs=inputs, labels=labels)

In [6]:
perceptron.weights

array([2., 0., 0., 0.])

## 3. Multilayer Perceptron <a id="Q3"></a>

Implement a Neural Network Multilayer Perceptron class that uses backpropagation to update the network's weights.
Your network must have one hidden layer.
You do not have to update weights via gradient descent. You can use something like the derivative of the sigmoid function to update weights.
Train your model on the Heart Disease dataset from UCI:



In [12]:
import pandas as pd
df = pd.read_csv('https://github.com/ryanleeallred/datasets/raw/master/heart.csv')

In [13]:
from sklearn.preprocessing import StandardScaler
X = df.drop('target',axis=1).values
X = StandardScaler().fit_transform(X)
y = df['target'].values.reshape([-1,1])

In [14]:
X.shape, y.shape

((303, 13), (303, 1))

In [15]:
class Neural_Network(object):
  def __init__(self):
    self.inputs = 13
    self.hiddenNodes = 1
    self.outputNodes = 1

    # Initlize Weights
    self.L1_weights = np.random.randn(self.inputs, self.hiddenNodes)
    self.L2_weights = np.random.randn(self.hiddenNodes, self.outputNodes)

  def feed_forward(self, X):
    # Weighted sum between inputs and hidden layer
    self.hidden_sum = np.dot(X, self.L1_weights)
    # Activations of weighted sum
    self.activated_hidden = self.sigmoid(self.hidden_sum)
    # Weighted sum between hidden and output
    self.output_sum = np.dot(self.activated_hidden, self.L2_weights)
    # final activation of output
    self.activated_output = self.sigmoid(self.output_sum)
    return self.activated_output
    
  def sigmoid(self, s):
    return 1/(1+np.exp(-s))
  
  def sigmoidPrime(self, s):
    return s * (1 - s)
  
  def backward(self, X, y, o):
    # backward propgate through the network
    self.o_error = y - o # error in output
    self.o_delta = self.o_error*self.sigmoidPrime(o) # applying derivative of sigmoid to error

    self.z2_error = self.o_delta.dot(self.L2_weights.T) # z2 error: how much our hidden layer weights contributed to output error
    self.z2_delta = self.z2_error*self.sigmoidPrime(self.activated_hidden) # applying derivative of sigmoid to z2 error

    self.L1_weights += X.T.dot(self.z2_delta) # adjusting first set (input --> hidden) weights
    self.L2_weights += self.activated_hidden.T.dot(self.o_delta) # adjusting second set (hidden --> output) weights
    
  def train (self, X, y):
    o = self.feed_forward(X)
    self.backward(X, y, o)

In [16]:
NN = Neural_Network()
for i in range(1000): # train the network 100 times
  if i+1 in [1,2,3,4,5] or (i+1) % 50 == 0:
    print("Loss: \n" + str(np.mean(np.square(y - NN.feed_forward(X))))) # mean sum squared loss
  NN.train(X, y)

Loss: 
0.34965511334084637
Loss: 
0.49392017168048447
Loss: 
0.4857647473540993
Loss: 
0.4766228588637929
Loss: 
0.4705660744597714
Loss: 
0.18313705475928896
Loss: 
0.17933900995411933
Loss: 
0.17910248209234356
Loss: 
0.17894253156037984
Loss: 
0.17877676306286214
Loss: 
0.17854986157359373
Loss: 
0.17843110551322183
Loss: 
0.17833292870161174
Loss: 
0.1782482143469527
Loss: 
0.17817394282323026
Loss: 
0.17810805694381543
Loss: 
0.17804904542803604
Loss: 
0.17799576285007854
Loss: 
0.17794732122132356
Loss: 
0.1779030191763284
Loss: 
0.1778622938258797
Loss: 
0.17782468705658647
Loss: 
0.1777898213781129
Loss: 
0.1777573822628704
Loss: 
0.17772710500734887


In [17]:
from sklearn.neural_network import MLPClassifier

# Instantiate multilayer perceptron class (alternative implementation)
mlp = MLPClassifier(hidden_layer_sizes=(1,), activation='logistic', # one hidden layer
                    learning_rate_init=0.05, solver='sgd', max_iter=1000,
                    batch_size=20, momentum=0.5)
# Fit and score it
mlp.fit(X, y)
mlp.score(X, y)

/Users/khaledadad/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:921: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8778877887788779

## 4. Keras MMP <a id="Q4"></a>

Implement a Multilayer Perceptron architecture of your choosing using the Keras library. Train your model and report its baseline accuracy. Then hyperparameter tune at least two parameters and report your model's accuracy.
Use the Heart Disease Dataset (binary classification)
Use an appropriate loss function for a binary classification task
Use an appropriate activation function on the final layer of your network.
Train your model using verbose output for ease of grading.
Use GridSearchCV to hyperparameter tune your model. (for at least two hyperparameters)
When hyperparameter tuning, show you work by adding code cells for each new experiment.
Report the accuracy for each combination of hyperparameters as you test them so that we can easily see which resulted in the highest accuracy.
You must hyperparameter tune at least 5 parameters in order to get a 3 on this section.

In [19]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

Using TensorFlow backend.


In [20]:
from sklearn.preprocessing import RobustScaler

In [21]:
scaler = RobustScaler()
X = scaler.fit_transform(data.iloc[:,:-1])
y = data.iloc[:,-1]

In [22]:
seed = 42
np.random.seed(seed)

def create_model():
    model = Sequential()
    model.add(Dense(12, input_dim=13, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, verbose=1)

kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=seed)

batch_size = [20,50,100]
epochs = [20,50,100]
param_grid = dict(batch_size=batch_size, epochs=epochs)

grid = GridSearchCV(estimator=model, param_grid=param_grid, 
                    n_jobs=1, cv=kfold)

grid_result = grid.fit(X, y)

W0816 12:09:12.475619 4408907200 deprecation_wrapper.py:119] From /Users/khaledadad/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0816 12:09:12.513925 4408907200 deprecation_wrapper.py:119] From /Users/khaledadad/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0816 12:09:12.520256 4408907200 deprecation_wrapper.py:119] From /Users/khaledadad/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0816 12:09:12.572129 4408907200 deprecation_wrapper.py:119] From /Users/khaledadad/anaconda3/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0816 12

Epoch 1/20
202/202 [==============================] - 0s 2ms/step - loss: 0.7655 - acc: 0.5000
Epoch 2/20
202/202 [==============================] - 0s 277us/step - loss: 0.7367 - acc: 0.5396
Epoch 3/20
202/202 [==============================] - 0s 218us/step - loss: 0.7119 - acc: 0.5990
Epoch 4/20
202/202 [==============================] - 0s 187us/step - loss: 0.6886 - acc: 0.6386
Epoch 5/20
202/202 [==============================] - 0s 167us/step - loss: 0.6659 - acc: 0.6386
Epoch 6/20
202/202 [==============================] - 0s 153us/step - loss: 0.6450 - acc: 0.6584
Epoch 7/20
202/202 [==============================] - 0s 136us/step - loss: 0.6271 - acc: 0.6782
Epoch 8/20
202/202 [==============================] - 0s 172us/step - loss: 0.6102 - acc: 0.7030
Epoch 9/20
202/202 [==============================] - 0s 126us/step - loss: 0.5963 - acc: 0.7129
Epoch 10/20
202/202 [==============================] - 0s 147us/step - loss: 0.5831 - acc: 0.7129
Epoch 11/20
202/202 [==========

202/202 [==============================] - 0s 152us/step - loss: 0.4342 - acc: 0.8416
Epoch 23/50
202/202 [==============================] - 0s 135us/step - loss: 0.4283 - acc: 0.8416
Epoch 24/50
202/202 [==============================] - 0s 175us/step - loss: 0.4227 - acc: 0.8416
Epoch 25/50
202/202 [==============================] - 0s 127us/step - loss: 0.4177 - acc: 0.8416
Epoch 26/50
202/202 [==============================] - 0s 174us/step - loss: 0.4122 - acc: 0.8465
Epoch 27/50
202/202 [==============================] - 0s 250us/step - loss: 0.4078 - acc: 0.8416
Epoch 28/50
202/202 [==============================] - 0s 234us/step - loss: 0.4030 - acc: 0.8416
Epoch 29/50
202/202 [==============================] - 0s 217us/step - loss: 0.3988 - acc: 0.8564
Epoch 30/50
202/202 [==============================] - 0s 185us/step - loss: 0.3950 - acc: 0.8564
Epoch 31/50
202/202 [==============================] - 0s 188us/step - loss: 0.3919 - acc: 0.8564
Epoch 32/50
202/202 [===========

202/202 [==============================] - 0s 604us/step - loss: 0.6345 - acc: 0.6683
Epoch 5/50
202/202 [==============================] - 0s 95us/step - loss: 0.6126 - acc: 0.7079
Epoch 6/50
202/202 [==============================] - 0s 94us/step - loss: 0.5937 - acc: 0.7327
Epoch 7/50
202/202 [==============================] - 0s 92us/step - loss: 0.5757 - acc: 0.7426
Epoch 8/50
202/202 [==============================] - 0s 87us/step - loss: 0.5581 - acc: 0.7673
Epoch 9/50
202/202 [==============================] - 0s 92us/step - loss: 0.5425 - acc: 0.7673
Epoch 10/50
202/202 [==============================] - 0s 107us/step - loss: 0.5283 - acc: 0.7772
Epoch 11/50
202/202 [==============================] - 0s 89us/step - loss: 0.5156 - acc: 0.7871
Epoch 12/50
202/202 [==============================] - 0s 83us/step - loss: 0.5036 - acc: 0.7921
Epoch 13/50
202/202 [==============================] - 0s 91us/step - loss: 0.4924 - acc: 0.8020
Epoch 14/50
202/202 [========================

202/202 [==============================] - 0s 121us/step - loss: 0.4173 - acc: 0.8416
Epoch 38/100
202/202 [==============================] - 0s 191us/step - loss: 0.4125 - acc: 0.8416
Epoch 39/100
202/202 [==============================] - 0s 105us/step - loss: 0.4082 - acc: 0.8416
Epoch 40/100
202/202 [==============================] - 0s 92us/step - loss: 0.4041 - acc: 0.8366
Epoch 41/100
202/202 [==============================] - 0s 132us/step - loss: 0.4001 - acc: 0.8416
Epoch 42/100
202/202 [==============================] - 0s 158us/step - loss: 0.3962 - acc: 0.8416
Epoch 43/100
202/202 [==============================] - 0s 143us/step - loss: 0.3925 - acc: 0.8465
Epoch 44/100
202/202 [==============================] - 0s 170us/step - loss: 0.3892 - acc: 0.8465
Epoch 45/100
202/202 [==============================] - 0s 208us/step - loss: 0.3856 - acc: 0.8465
Epoch 46/100
202/202 [==============================] - 0s 110us/step - loss: 0.3826 - acc: 0.8465
Epoch 47/100
202/202 [==

...

202/202 [==============================] - 0s 45us/step - loss: 0.6280 - acc: 0.6436
Epoch 10/100
202/202 [==============================] - 0s 61us/step - loss: 0.6237 - acc: 0.6485
Epoch 11/100
202/202 [==============================] - 0s 58us/step - loss: 0.6196 - acc: 0.6535
Epoch 12/100
202/202 [==============================] - 0s 54us/step - loss: 0.6154 - acc: 0.6683
Epoch 13/100
202/202 [==============================] - 0s 47us/step - loss: 0.6110 - acc: 0.6782
Epoch 14/100
202/202 [==============================] - 0s 68us/step - loss: 0.6068 - acc: 0.6782
Epoch 15/100
202/202 [==============================] - 0s 102us/step - loss: 0.6028 - acc: 0.6931
Epoch 16/100
202/202 [==============================] - 0s 103us/step - loss: 0.5990 - acc: 0.6980
Epoch 17/100
202/202 [==============================] - 0s 86us/step - loss: 0.5954 - acc: 0.6980
Epoch 18/100
202/202 [==============================] - 0s 92us/step - loss: 0.5921 - acc: 0.6980
Epoch 19/100
202/202 [=========

202/202 [==============================] - 0s 84us/step - loss: 0.4364 - acc: 0.8218
Epoch 93/100
202/202 [==============================] - 0s 49us/step - loss: 0.4348 - acc: 0.8119
Epoch 94/100
202/202 [==============================] - 0s 71us/step - loss: 0.4335 - acc: 0.8119
Epoch 95/100
202/202 [==============================] - 0s 58us/step - loss: 0.4320 - acc: 0.8119
Epoch 96/100
202/202 [==============================] - 0s 71us/step - loss: 0.4307 - acc: 0.8119
Epoch 97/100
202/202 [==============================] - 0s 52us/step - loss: 0.4293 - acc: 0.8119
Epoch 98/100
202/202 [==============================] - 0s 74us/step - loss: 0.4280 - acc: 0.8119
Epoch 99/100
202/202 [==============================] - 0s 60us/step - loss: 0.4270 - acc: 0.8168
Epoch 100/100
101/101 [==============================] - 0s 4ms/step
Epoch 1/100
202/202 [==============================] - 1s 7ms/step - loss: 0.6970 - acc: 0.5347
Epoch 2/100
202/202 [==============================] - 0s 75us/s

202/202 [==============================] - 0s 43us/step - loss: 0.4787 - acc: 0.8267
Epoch 76/100
202/202 [==============================] - 0s 64us/step - loss: 0.4770 - acc: 0.8317
Epoch 77/100
202/202 [==============================] - 0s 43us/step - loss: 0.4751 - acc: 0.8317
Epoch 78/100
202/202 [==============================] - 0s 72us/step - loss: 0.4731 - acc: 0.8366
Epoch 79/100
202/202 [==============================] - 0s 56us/step - loss: 0.4710 - acc: 0.8366
Epoch 80/100
202/202 [==============================] - 0s 66us/step - loss: 0.4693 - acc: 0.8416
Epoch 81/100
202/202 [==============================] - 0s 88us/step - loss: 0.4675 - acc: 0.8416
Epoch 82/100
202/202 [==============================] - 0s 72us/step - loss: 0.4659 - acc: 0.8416
Epoch 83/100
202/202 [==============================] - 0s 83us/step - loss: 0.4642 - acc: 0.8416
Epoch 84/100
202/202 [==============================] - 0s 87us/step - loss: 0.4628 - acc: 0.8416
Epoch 85/100
202/202 [===========

202/202 [==============================] - 0s 41us/step - loss: 0.5108 - acc: 0.8020
Epoch 59/100
202/202 [==============================] - 0s 42us/step - loss: 0.5088 - acc: 0.8020
Epoch 60/100
202/202 [==============================] - 0s 43us/step - loss: 0.5068 - acc: 0.7921
Epoch 61/100
202/202 [==============================] - 0s 38us/step - loss: 0.5047 - acc: 0.7921
Epoch 62/100
202/202 [==============================] - 0s 38us/step - loss: 0.5025 - acc: 0.7921
Epoch 63/100
202/202 [==============================] - 0s 38us/step - loss: 0.5004 - acc: 0.7921
Epoch 64/100
202/202 [==============================] - 0s 39us/step - loss: 0.4985 - acc: 0.7921
Epoch 65/100
202/202 [==============================] - 0s 43us/step - loss: 0.4965 - acc: 0.7921
Epoch 66/100
202/202 [==============================] - 0s 49us/step - loss: 0.4946 - acc: 0.7921
Epoch 67/100
202/202 [==============================] - 0s 41us/step - loss: 0.4926 - acc: 0.7921
Epoch 68/100
202/202 [===========

303/303 [==============================] - 0s 53us/step - loss: 0.4409 - acc: 0.8284
Epoch 42/100
303/303 [==============================] - 0s 55us/step - loss: 0.4364 - acc: 0.8251
Epoch 43/100
303/303 [==============================] - 0s 59us/step - loss: 0.4322 - acc: 0.8317
Epoch 44/100
303/303 [==============================] - 0s 59us/step - loss: 0.4284 - acc: 0.8350
Epoch 45/100
303/303 [==============================] - 0s 56us/step - loss: 0.4241 - acc: 0.8317
Epoch 46/100
303/303 [==============================] - 0s 65us/step - loss: 0.4203 - acc: 0.8317
Epoch 47/100
303/303 [==============================] - 0s 66us/step - loss: 0.4165 - acc: 0.8317
Epoch 48/100
303/303 [==============================] - 0s 52us/step - loss: 0.4132 - acc: 0.8350
Epoch 49/100
303/303 [==============================] - 0s 56us/step - loss: 0.4101 - acc: 0.8383
Epoch 50/100
303/303 [==============================] - 0s 57us/step - loss: 0.4069 - acc: 0.8383
Epoch 51/100
303/303 [===========

In [23]:
print(f"Best: {grid_result.best_score_:.3f} using {grid_result.best_params_}")

Best: 0.828 using {'batch_size': 50, 'epochs': 100}


In [24]:
seed = 42
np.random.seed(seed)

def create_model():
    model = Sequential()
    model.add(Dense(12, input_dim=13, activation='relu'))
    model.add(Dense(12, input_dim=13, activation='relu'))
    model.add(Dense(12, input_dim=13, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, verbose=1)

kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=seed)

batch_size = [20]
epochs = [100]
#optimizer = ['adam','sgd','rmsprop','nadam']

param_grid = dict(batch_size=batch_size, 
                  epochs=epochs
                 #,optimizer=optimizer
                 )

grid = GridSearchCV(estimator=model, param_grid=param_grid, 
                    n_jobs=1, cv=kfold)

grid_result = grid.fit(X, y)

Epoch 1/100
202/202 [==============================] - 2s 8ms/step - loss: 0.6809 - acc: 0.5693
Epoch 2/100
202/202 [==============================] - 0s 176us/step - loss: 0.6584 - acc: 0.6139
Epoch 3/100
202/202 [==============================] - 0s 188us/step - loss: 0.6396 - acc: 0.6931
Epoch 4/100
202/202 [==============================] - 0s 201us/step - loss: 0.6213 - acc: 0.7277
Epoch 5/100
202/202 [==============================] - 0s 181us/step - loss: 0.6038 - acc: 0.7624
Epoch 6/100
202/202 [==============================] - 0s 202us/step - loss: 0.5836 - acc: 0.7624
Epoch 7/100
202/202 [==============================] - 0s 198us/step - loss: 0.5624 - acc: 0.7723
Epoch 8/100
202/202 [==============================] - 0s 197us/step - loss: 0.5405 - acc: 0.7772
Epoch 9/100
202/202 [==============================] - 0s 209us/step - loss: 0.5205 - acc: 0.7772
Epoch 10/100
202/202 [==============================] - 0s 178us/step - loss: 0.5012 - acc: 0.7871
Epoch 11/100
202/202 

202/202 [==============================] - 0s 466us/step - loss: 0.1655 - acc: 0.9356
Epoch 84/100
202/202 [==============================] - 0s 260us/step - loss: 0.1633 - acc: 0.9406
Epoch 85/100
202/202 [==============================] - 0s 247us/step - loss: 0.1640 - acc: 0.9356
Epoch 86/100
202/202 [==============================] - 0s 251us/step - loss: 0.1606 - acc: 0.9406
Epoch 87/100
202/202 [==============================] - 0s 204us/step - loss: 0.1583 - acc: 0.9406
Epoch 88/100
202/202 [==============================] - 0s 263us/step - loss: 0.1567 - acc: 0.9406
Epoch 89/100
202/202 [==============================] - 0s 229us/step - loss: 0.1557 - acc: 0.9356
Epoch 90/100
202/202 [==============================] - 0s 250us/step - loss: 0.1553 - acc: 0.9406
Epoch 91/100
202/202 [==============================] - 0s 237us/step - loss: 0.1517 - acc: 0.9455
Epoch 92/100
202/202 [==============================] - 0s 257us/step - loss: 0.1490 - acc: 0.9455
Epoch 93/100
202/202 [=

202/202 [==============================] - 0s 175us/step - loss: 0.2779 - acc: 0.9059
Epoch 66/100
202/202 [==============================] - 0s 138us/step - loss: 0.2758 - acc: 0.9059
Epoch 67/100
202/202 [==============================] - 0s 139us/step - loss: 0.2736 - acc: 0.9059
Epoch 68/100
202/202 [==============================] - 0s 157us/step - loss: 0.2724 - acc: 0.9109
Epoch 69/100
202/202 [==============================] - 0s 143us/step - loss: 0.2709 - acc: 0.9158
Epoch 70/100
202/202 [==============================] - 0s 169us/step - loss: 0.2705 - acc: 0.9158
Epoch 71/100
202/202 [==============================] - 0s 185us/step - loss: 0.2676 - acc: 0.9109
Epoch 72/100
202/202 [==============================] - 0s 168us/step - loss: 0.2678 - acc: 0.9109
Epoch 73/100
202/202 [==============================] - 0s 273us/step - loss: 0.2643 - acc: 0.9109
Epoch 74/100
202/202 [==============================] - 0s 245us/step - loss: 0.2629 - acc: 0.9109
Epoch 75/100
202/202 [=

202/202 [==============================] - 0s 145us/step - loss: 0.2717 - acc: 0.9059
Epoch 48/100
202/202 [==============================] - 0s 142us/step - loss: 0.2690 - acc: 0.9109
Epoch 49/100
202/202 [==============================] - 0s 148us/step - loss: 0.2658 - acc: 0.9109
Epoch 50/100
202/202 [==============================] - 0s 137us/step - loss: 0.2612 - acc: 0.9059
Epoch 51/100
202/202 [==============================] - 0s 143us/step - loss: 0.2578 - acc: 0.9059
Epoch 52/100
202/202 [==============================] - 0s 147us/step - loss: 0.2553 - acc: 0.9059
Epoch 53/100
202/202 [==============================] - 0s 148us/step - loss: 0.2537 - acc: 0.9208
Epoch 54/100
202/202 [==============================] - 0s 157us/step - loss: 0.2498 - acc: 0.9208
Epoch 55/100
202/202 [==============================] - 0s 155us/step - loss: 0.2458 - acc: 0.9158
Epoch 56/100
202/202 [==============================] - 0s 134us/step - loss: 0.2423 - acc: 0.9158
Epoch 57/100
202/202 [=

303/303 [==============================] - 0s 186us/step - loss: 0.3027 - acc: 0.8746
Epoch 30/100
303/303 [==============================] - 0s 222us/step - loss: 0.3014 - acc: 0.8746
Epoch 31/100
303/303 [==============================] - 0s 220us/step - loss: 0.2986 - acc: 0.8746
Epoch 32/100
303/303 [==============================] - 0s 214us/step - loss: 0.2947 - acc: 0.8746
Epoch 33/100
303/303 [==============================] - 0s 151us/step - loss: 0.2913 - acc: 0.8779
Epoch 34/100
303/303 [==============================] - 0s 152us/step - loss: 0.2892 - acc: 0.8779
Epoch 35/100
303/303 [==============================] - 0s 126us/step - loss: 0.2878 - acc: 0.8812
Epoch 36/100
303/303 [==============================] - 0s 124us/step - loss: 0.2842 - acc: 0.8779
Epoch 37/100
303/303 [==============================] - 0s 147us/step - loss: 0.2828 - acc: 0.8779
Epoch 38/100
303/303 [==============================] - 0s 129us/step - loss: 0.2783 - acc: 0.8845
Epoch 39/100
303/303 [=

In [25]:
print(f"Best: {grid_result.best_score_:.3f} using {grid_result.best_params_}")

Best: 0.812 using {'batch_size': 20, 'epochs': 100}
